In [1]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd

from pymatgen.transformations.standard_transformations import SupercellTransformation

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

import os
#os.getcwd()
#os.chdir('data')

In [2]:
fe_cif_filepath = "data/fe.cif"
fe_structure_2atom = from_file(structure_file=fe_cif_filepath)
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)
fe_structure_4atom = rotate_cell_45_degrees.apply_transformation(fe_structure_2atom)
print(fe_structure_4atom)

Full Formula (Fe4)
Reduced Formula: Fe
abc   :   4.053843   4.053843   2.866500
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP      a    b     c
---  ----  ---  ---  ----
  0  Fe    0    0     0
  1  Fe    0.5  0.5  -0
  2  Fe    0.5  0     0.5
  3  Fe    1    0.5   0.5


Use the 4 atom basis to generate the n = NxNxN supercell configuration. 
The neighbor information is extracted from the results of *count_neighbors* . The final output is the total number of sites,  4n  , the count of nearest neighbors for each site within the radius, r , and a list of the indices for the nearest neighbors of each site _i_.

This information can be used as the input to the Monte Carlo Simulation, together with the calculated exchange parameters. 

In [3]:
N = 8
r = 3
fe_structure_supercell = fe_structure_4atom.copy()
fe_structure_supercell.make_supercell([N,N,N])
fe_neighbor_data = count_neighbors(fe_structure_supercell, r)
a = fe_neighbor_data.data_frame \
    .merge(fe_neighbor_data.data_frame.rename(columns={"j": "i", "i": "j"}), how="outer") \
    .sort_values(["i","distance_bin", "j"]) \
    .loc[:, ["i", "j", "distance_bin"]] \
    .reset_index(drop=True)

In [4]:
neighbors_per_site = a.groupby(["i"]).count().reset_index().rename(columns={"j": "count"}).loc[:, "count"].values
sites_per_distance_group = a.groupby(["i", "distance_bin"]).count().reset_index() \
                            .rename(columns={"j": "count"}).loc[:, "count"].values
neighbor_indices = a["j"].values

In [5]:
#neighbor_count, n_indices = np.unique(sites_per_distance_group, return_index=True)
neighbor_count = np.dstack ( np.unique(sites_per_distance_group, return_index=True) )    
neighbor_count.dtype = np.dtype([('v', neighbor_count.dtype), ('i', neighbor_count.dtype)])
neighbor_count.sort(order='i', axis=1)
neighbor_count = neighbor_count.flatten()['v'].tolist()

To be used in the MC simulation:

In [6]:
print("Number of sites:", 4*N*N*N)
print("Number of 1st and 2nd neighbors for each index i:", neighbor_count)
print("Neighbor indices sorted over index i and distances: ", neighbor_indices)

Number of sites: 2048
Number of 1st and 2nd neighbors for each index i: [8, 6]
Neighbor indices sorted over index i and distances:  [1024 1031 1472 ... 1535 2040 2046]


In [7]:
filename = os.getcwd()+'/neighbor_lists.txt'

n_sites = np.array([4*N*N*N])
#np.savetxt(filename,n_sites,fmt='%d' )
#np.savetxt(filename,neighbor_count,fmt='%d' )
f=open(filename,'w')
#with open(filename,"w") as f:
f.write("\n".join(" ".join(map(str, x)) for x in (n_sites,neighbor_count)))
f.write("\n")
#f.write(str(neighbor_indices))
#f.close()
#f=open(filename,'a')
np.savetxt(filename, neighbor_indices,fmt='%d')
f.close()


In [8]:
#os.getcwd()
#os.chdir('/home/ssilayi/Projects/spyns/Fe/exchange_model')

In [9]:
fe_structure_supercell

Structure Summary
Lattice
    abc : 32.43074541233982 32.43074541233981 22.932
 angles : 90.0 90.0 90.0
 volume : 24118.805483136
      A : 22.932000000000002 22.932 2.80836003980471e-15
      B : 22.931999999999995 -22.932 0.0
      C : 0.0 0.0 22.932
PeriodicSite: Fe (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Fe (0.0000, 0.0000, 2.8665) [0.0000, 0.0000, 0.1250]
PeriodicSite: Fe (0.0000, 0.0000, 5.7330) [0.0000, 0.0000, 0.2500]
PeriodicSite: Fe (0.0000, 0.0000, 8.5995) [0.0000, 0.0000, 0.3750]
PeriodicSite: Fe (0.0000, 0.0000, 11.4660) [0.0000, 0.0000, 0.5000]
PeriodicSite: Fe (0.0000, 0.0000, 14.3325) [0.0000, 0.0000, 0.6250]
PeriodicSite: Fe (0.0000, 0.0000, 17.1990) [0.0000, 0.0000, 0.7500]
PeriodicSite: Fe (0.0000, 0.0000, 20.0655) [0.0000, 0.0000, 0.8750]
PeriodicSite: Fe (25.7985, 20.0655, 22.9320) [1.0000, 0.1250, 1.0000]
PeriodicSite: Fe (25.7985, 20.0655, 2.8665) [1.0000, 0.1250, 0.1250]
PeriodicSite: Fe (25.7985, 20.0655, 5.7330) [1.0000, 0.1250, 0.2500]